In [ ]:
!pip3 install torch torchvision

In [88]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

C:\Users\mytra\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe
3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
sys.version_info(major=3, minor=11, micro=9, releaselevel='final', serial=0)


In [91]:
import numpy as np 
import pandas as pd
import os
import torch as t 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [92]:
# read the training data
df_data = pd.read_csv('CarPrice_Assignment.csv')

df_data.head()

,car_ID,enginesize,horsepower,citympg,highwaympg,price
0,1,130,111,21,27,13495.0
1,2,130,111,21,27,16500.0
2,3,152,154,19,26,16500.0
3,4,109,102,24,30,13950.0
4,5,136,115,18,22,17450.0


In [93]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   car_ID      205 non-null    int64  
 1   enginesize  205 non-null    int64  
 2   horsepower  205 non-null    int64  
 3   citympg     205 non-null    int64  
 4   highwaympg  205 non-null    int64  
 5   price       205 non-null    float64
dtypes: float64(1), int64(5)
memory usage: 9.7 KB


In [94]:

df_data.describe()  #looking at the file, we are only focusing on the engine size, horsepower, citympg, highway mpg

,car_ID,enginesize,horsepower,citympg,highwaympg,price
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,103.000000,126.907317,104.117073,25.219512,30.751220,13276.710571
std,59.322565,41.642693,39.544167,6.542142,6.886443,7988.852332
min,1.000000,61.000000,48.000000,13.000000,16.000000,5118.000000
25%,52.000000,97.000000,70.000000,19.000000,25.000000,7788.000000
50%,103.000000,120.000000,95.000000,24.000000,30.000000,10295.000000
75%,154.000000,141.000000,116.000000,30.000000,34.000000,16503.000000
max,205.000000,326.000000,288.000000,49.000000,54.000000,45400.000000


In [95]:
y_np = df_data['price'].to_numpy()
y_np.shape

(205,)

In [96]:
X_np = df_data[['enginesize','horsepower', 'citympg', 'highwaympg']].to_numpy()  #we are only looking at these features 

df_data[['enginesize','horsepower', 'citympg', 'highwaympg', 'price']].corr()  #now we want to inlcude the price to see how they correlate 

,enginesize,horsepower,citympg,highwaympg,price
enginesize,1.000000,0.809769,-0.653658,-0.677470,0.874145
horsepower,0.809769,1.000000,-0.801456,-0.770544,0.808139
citympg,-0.653658,-0.801456,1.000000,0.971337,-0.685751
highwaympg,-0.677470,-0.770544,0.971337,1.000000,-0.697599
price,0.874145,0.808139,-0.685751,-0.697599,1.000000


In [97]:
df_corr=df_data.corr()
select_columns=df_corr['price'][np.abs(df_corr['price'])>=0.03]
print(select_columns)
select_columns=select_columns.index.to_list()[1:]
print(select_columns)

car_ID       -0.109093
enginesize    0.874145
horsepower    0.808139
citympg      -0.685751
highwaympg   -0.697599
price         1.000000
Name: price, dtype: float64
['enginesize', 'horsepower', 'citympg', 'highwaympg', 'price']


In [98]:
y_np = df_data['price'].to_numpy()
print(y_np.shape)
x_np = df_data[select_columns].to_numpy()
print(x_np.shape)

(205,)
(205, 5)


In [99]:
corr_with_output = abs(df_corr['price']).sort_values(ascending=False)[1:]
selected_features = corr_with_output[corr_with_output >= 0.2].index

# Check for high intercorrelation among the selected features
selected_corr = df_corr.loc[selected_features, selected_features]
drop_features = [col for col in selected_features if any(selected_corr[col].abs() >= 0.8)]

# Drop the highly intercorrelated features
selected_features = [feat for feat in selected_features if feat not in drop_features]

# Extract the selected features
X_np = df_data[selected_features].to_numpy()


In [100]:
import sklearn 
from sklearn.model_selection import train_test_split
import torch as t 

X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(x_np, y_np, test_size = 0.2, random_state = 42)


from sklearn.preprocessing import StandardScaler


x_scaler = StandardScaler()
x_scaler = x_scaler.fit(X_train_np)
# print(x_scaler)
X_train_norm_np = x_scaler.transform(X_train_np)
X_test_norm_np  = x_scaler.transform(X_test_np)
#X_test_norm_transform(X_train_np)

y_scaler = StandardScaler()
y_scaler = y_scaler.fit(y_train_np.reshape(-1,1))
print(y_scaler.mean_, y_scaler.var_)
y_train_norm_np = y_scaler.transform(y_train_np.reshape(-1,1))
y_test_norm_np  = y_scaler.transform(y_test_np.reshape(-1,1))
#y_train_norm_np =

ModuleNotFoundError: No module named 'sklearn'

In [104]:
# convert to tensors (pytorch)
# convert data from numpy to tensors
X_t_train = t.from_numpy(X_train_norm_np).float()
y_t_train = t.flatten(t.from_numpy(y_train_norm_np).float()) # flatten - creates a one dimensional tensor
X_t_test   = t.from_numpy(X_test_norm_np).float()
y_t_test   = t.flatten(t.from_numpy(y_test_norm_np).float())

print(X_t_train.shape, y_t_train.shape)
print(X_t_train.dtype, y_t_train.dtype)
print(X_t_test.shape,   y_t_test.shape)
print(X_t_test.dtype,   y_t_test.dtype)

NameError: name 't' is not defined

In [ ]:
import torch.nn.functional as F
import torch.nn as nn
from   torch.autograd import Variable

class Model(nn.Module):
    def __init__(self, input_dim = 5):
        super(Model, self).__init__()
        self.layer1   = nn.Linear(in_features = input_dim, out_features = 50)
        #self.dropout1 = nn.Dropout(p = 0.3) # drop 30% of output nodes from the previous layer during training only
        self.layer2   = nn.Linear(in_features= 50, out_features = 20)
        #self.dropout2 = nn.Dropout(p = 0.25)
        self.layer3   = nn.Linear(in_features = 20, out_features = 1) # 3 neurons = one for each class


    def forward(self, x): # x input vector of size 7
        x = F.relu(self.layer1(x)) # x = vector of size 50
        #x = self.dropout1(x)
        x = F.relu(self.layer2(x)) #
        #x = self.dropout2(x)
        x = self.layer3(x) # x = one value linear output neuron
        return x

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\mytra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [ ]:
# create a Model
model = Model(input_dim = 5)
print(model)

NameError: name 'Model' is not defined

In [ ]:
test_out = model(X_t_train[:5,:])
print(test_out, y_t_train[:5], test_out.shape)

NameError: name 'model' is not defined

In [105]:
# write the training loop:
def train(model, optim, loss, train_loader, test_loader, n_epochs = 5):
    for e in range(n_epochs):

          total_train_loss = 0
          # set model in learning mode
          model.train()
          # for each batch in train_loader
          for xb,yb in train_loader:
        #    forward pass
               y_model = model.forward(xb)
        #    compute the loss on the batch output
               loss_batch = loss(y_model.flatten(),yb)
               total_train_loss += loss_batch
        #    compute gradients
               optim.zero_grad() # sets gradients to 0
               loss_batch.backward() # compute gradients
        #    backward pass - updates the weights (parameters) of the model
               optim.step()
        #    backward pass - updates the weights (parameters) of the model
        # set model in test mode

          total_test_loss = 0.0
        # for each batch in test_loader:
          model.eval() # set model in evaluation mode = no dropout
          with t.no_grad():
               for xt, yt in test_loader:
        #    forward pass
                    y_model = model.forward(xt)
        #    compute the loss on the batch
                    loss_batch = loss(y_model.flatten(), yt)
                    total_test_loss += loss_batch

          print("Epoch = ", e, "\tTrain loss = ", total_train_loss, "\tTest loss = ", total_test_loss)


In [107]:
# prepare the data loaders
# DataLoader = a class that shuffles the data and splits in into batches
# you should use it during training (SGD - accumulate error over batches of data )
train_data = [(X_t_train[i], y_t_train[i]) for i in range(X_t_train.shape[0])]
print("Sample train_data = ", train_data[:3], " type = ", type(train_data))
train_loader = t.utils.data.DataLoader(train_data, batch_size = 32, shuffle=True)
for x,label in train_loader:  # shuffles the data
     print(x.shape,label.shape)
     break
#print(train_data[0][0].shape)
test_data = [(X_t_test[i], y_t_test[i]) for i in range(X_t_test.shape[0])]
#print("Sample train_data = ", test_data[:3], " type = ", type(test_data))
test_loader = t.utils.data.DataLoader(test_data, batch_size = 32, shuffle=True)

NameError: name 'X_t_train' is not defined

In [108]:
y_t_train.shape

# train the model
model1 = Model(input_dim = 7)
optim  = t.optim.Adam(model1.parameters(), lr = 0.)
loss   = nn.MSELoss()

train(model, optim, loss, train_loader, test_loader, n_epochs = 10)

NameError: name 'y_t_train' is not defined

In [109]:
# extract numeric columns
df_numeric = df_data.select_dtypes(['int64'])
df_numeric.columns

Index(['car_ID', 'enginesize', 'horsepower', 'citympg', 'highwaympg'], dtype='object')